# Analyse potential persistent contrail formation as a function of the mixing line slope

_Dataset:_ Supplementary data for Megill and Grewe (2024): "Investigating the limiting aircraft design-dependent and environmental factors of persistent contrail formation".

_Authors:_

- Liam Megill (1, 2), https://orcid.org/0000-0002-4199-6962   
- Volker Grewe (1, 2), https://orcid.org/0000-0002-8012-6783  

_Affiliation (1)_: Deutsches Zentrum für Luft- und Raumfahrt (DLR), Institut für Physik der Atmosphäre, Oberpfaffenhofen, Germany

_Affiliation (2)_: Delft University of Technology (TU Delft), Faculty of Aerospace Engineering, Section Aircraft Noise and Climate Effects (ANCE), Delft, The Netherlands

_Corresponding author_: Liam Megill, liam.megill@dlr.de

_doi_: https://doi.org/10.5194/egusphere-2024-3398

---


### Summary
This notebook analyses the potential persistent contrail formation $p_{pcf}$ as a function of the mixing line slope $G$, pressure level and altitude. We calculate $p_{pcf}$ by cumulatively summing the histograms of the maximum mixing line slopes $G_{max}$ calculated in `15-lm-Gmax_grib.ipynb`. In this notebook, we create Figure 6 and Supplementary Figure 6 of the linked paper. The figure creation can take 10 minutes per figure.


### Inputs
- `data/processed/ppcf/ppcfhist_M_2010s_ERA5_GRIB_v2.nc`: Dataset of monthly histograms in the 2010 decade.
- `data/processed/ppcf/ppcfhist_S_2010s_ERA5_GRIB_v2.nc`: Dataset of seasonal histograms in the 2010 decade.

### Outputs
- Figure 6
- Supplementary Figure 6

---

### Copyright

Copyright © 2024 Liam Megill

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [ ]:
import xarray as xr
import numpy as np
import datetime
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
import seaborn as sns
from helper import calc_cum_hist, calc_ppcf_fits

# define directories
project_dir = ""  # set top-level directory path
processed_data_dir = project_dir + "data/processed/ppcf/"

# define figure variables
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']

# load data
ds_allM = xr.open_dataset(processed_data_dir+"ppcfhist_M_2010s_ERA5_GRIB_v2.nc")
ds_allS = xr.open_dataset(processed_data_dir+"ppcfhist_S_2010s_ERA5_GRIB_v2.nc")

ds_allS

The first step is to select a single RHi enhancement (here "correction") and compare the responses per latitude band (global, northern extratropics, tropics, southern extratropics). In Figure 6 of the linked paper, we use the uncorrected data (`corr` = "uncor"). The right side of that figure shows that there are two clearly defined responses, one for the extratropics and another for the tropics. For the global responses, we thus fit the data using a combination of two logistic functions, one representing the tropics and the other the extratropics.

We also calculate the mixing line slope $G$ at which the fits get to within 1 % of the supremum value. We see that this value of $G$ generally reduces with increasing altitude (decreasing pressure). This means that aircraft design becomes less relevant for persistent contrail formation with increasing altitude.

To save the figures, uncomment the last line and choose a saving location.

In [ ]:
corr = "uncor"  # select correction (one of "uncor", "RHi-cor-98p", "RHi-cor-95p", "RHi-cor-90p")
ds_allS_c = ds_allS.sel(corr=corr)  # dataset including only single correction

# initialise figure
fig = plt.figure(figsize=(9, 5.5))
gs = GridSpec(3, 3, figure=fig)
ax_tot = fig.add_subplot(gs[:, 0:2]); ax_nxtr = fig.add_subplot(gs[0, 2])
ax_tr = fig.add_subplot(gs[1, 2]); ax_sxtr = fig.add_subplot(gs[2, 2])
axs = [ax_tot, ax_nxtr, ax_tr, ax_sxtr]

# define latitude bands (match with those in dataset)
lat_bands = ["tot_hist", "xtropN_hist", "trop_hist", "xtropS_hist"]  # in this order!

# for loop over latitude band
for lat_band, ax in zip(lat_bands, axs):
    # calculate cumulative histograms
    new_bin_centres = np.arange(4.7, 6.7, 0.2)
    bin_centres, cum_hist = calc_cum_hist(ds_allS_c, lat_band, new_bin_centres)

    # plot seasonal cumulative histograms
    ax.axvline(4.5, color="gray", linestyle="-", linewidth=0.5, zorder=4)
    for i in range(len(ds_allS_c.season)):
        for i_lvl, lvl in enumerate(ds_allS_c.level):
            ax.plot(ds_allS_c.bin_centre.values,
                    cum_hist[i, i_lvl, :-len(new_bin_centres)],
                    color=colors[i_lvl], alpha=0.4, zorder=2)
            ax.plot(np.concatenate(([4.5], new_bin_centres)),
                    cum_hist[i, i_lvl, -len(new_bin_centres)-1:],
                    color=colors[i_lvl], alpha=0.1, zorder=2)

    # only calculate fit for tot_hist
    if lat_band == "tot_hist":
        threshold_vals = {}
        fit_res = calc_ppcf_fits(ds_allS.sel(corr="uncor"), cum_hist, bin_centres)
        x_data_plt = np.arange(min(bin_centres), max(bin_centres), 0.01)
        for i_lvl, lvl in enumerate(ds_allS_c.level.data):
            if i_lvl < 5:
                y1 = logistic_gen(x_data_plt, *fit_res[lvl]["params_1"])
                y2 = logistic(x_data_plt, *fit_res[lvl]["params_2"])
                y_comb = y1 + y2
            else:
                y_comb = logistic_gen(x_data_plt, *fit_res[lvl]["params"])
            # plot fits per level
            ax.plot(x_data_plt, y_comb, ls="--", c="k", label="", linewidth=1.0, zorder=3)

            # calculate 1% of threshold per level
            threshold = 0.99 * fit_res[lvl]["Lpd"]
            idx = np.where(y_comb >= threshold)[0][0]
            threshold_vals[lvl] = (x_data_plt[idx], y_comb[idx])
            ax.scatter(x_data_plt[idx], y_comb[idx], c="k", marker="o", s=8, zorder=5)

        # calculate and plot level-independent fit 
        y1t = logistic_gen(x_data_plt, *fit_res["all"]["params_1"])
        y2t = logistic(x_data_plt, *fit_res["all"]["params_2"])
        y_combt = y1t + y2t
        ax.plot(x_data_plt, y_combt, ls="-.", c="k", zorder=3)

        # calculate 1% of threshold for level-independent values
        threshold = 0.99 * fit_res["all"]["Lpd"]
        idx = np.where(y_combt >= threshold)[0][0]
        threshold_vals["all"] = (x_data_plt[idx], y_combt[idx])
        ax.scatter(x_data_plt[idx], y_combt[idx], c="k", marker="o", s=8, zorder=5)


# adjust axes and labels
ax_tot.set_xlabel("Mixing line slope $G$ [Pa/K]", fontsize=12)
ax_tot.set_ylabel("Potential persistent contrail formation $p_{PCF}$ [-]", fontsize=12)
ax_tot.set_ylim([0., 0.10])
ax_tot.set_xlim([-0.3, 6.7])

ax_nxtr.yaxis.tick_right()
ax_nxtr.set_yticks([0.0, 0.05, 0.10, 0.15])
ax_nxtr.set_ylim([-0.01, 0.15])
ax_nxtr.set_xlim([-0.3, 6.7])

ax_tr.set_ylabel("Potential persistent contrail formation $p_{PCF}$ [-]", fontsize=12)
ax_tr.yaxis.set_label_position("right")
ax_tr.yaxis.tick_right()
ax_tr.set_ylim([-0.01, 0.15])
ax_tr.set_yticks([0.0, 0.05, 0.10])
ax_tr.set_xlim([-0.3, 6.7])

ax_sxtr.yaxis.tick_right()
ax_sxtr.set_xlabel("Mixing line slope $G$ [Pa/K]", fontsize=12)
ax_sxtr.set_ylim([-0.01, 0.15])
ax_sxtr.set_yticks([0.0, 0.05, 0.10])
ax_sxtr.set_xlim([-0.3, 6.7])

# add subplot labels
fig.text(0.10, 0.83, "a", fontsize=18)
fig.text(0.65, 0.83, "b", fontsize=18)
fig.text(0.65, 0.58, "c", fontsize=18)
fig.text(0.65, 0.32, "d", fontsize=18)

# add aircraft labels
G_lst_250hPa = [0.48, 0.97, 1.15, 1.65, 1.93, 4.97, 5.27]
ax_tot2 = ax_tot.twiny()
ax_tot2.set_xticks(G_lst_250hPa)
ax_tot2.set_xticklabels(["WET-75", "WET-50", "HYB-80", "CON-LG", "CON-NG", "H2C-04", "H2FC-LV"],
                    rotation=45, ha="left", fontsize=8)
ax_tot2.set_xlim([-0.3, 6.7])

# create custom legend
legend_lines = [Line2D([0], [0], color=color, lw=2) for color in colors]
legend_lines.append(Line2D([0], [0], color="k", lw=1, ls="--"))
legend_lines.append(Line2D([0], [0], color="k", lw=1, ls="-."))
legend_labels = np.append(ds_allS_c.level.values, ["Fitted", "All data"])
ax_tot.legend(legend_lines, legend_labels, loc="lower right", ncol=2, handlelength=3, framealpha=1)

fig.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0)
# fig.savefig("figs/G-vs-ppcf_fitted_all.png", dpi=600)

Next, we analyse the influence of the RHi enhancements ("corrections"). In this study, we use simple factors 1/RHi_C with RHi_C = [1.0, 0.98, 0.95, 0.90]. As RHi is enhanced, more persistent contrails form. However, the general shape of the results stays the same.

To save the figure, uncomment the last line and choose a saving location.

In [ ]:
# figure with subplots
corr_lst = ["uncor", "RHi-cor-98p", "RHi-cor-95p", "RHi-cor-90p"]

# initialise figure
fig, ((ax_uncor, ax_98p), (ax_95p, ax_90p)) = plt.subplots(2, 2, figsize=(10, 7), sharex=True, sharey="row")
axs = [ax_uncor, ax_98p, ax_95p, ax_90p]

for corr, ax in zip(corr_lst, axs):
    # calculate cumulative histograms
    ds_allS_c = ds_allS.sel(corr=corr)
    new_bin_centres = np.arange(4.7, 6.7, 0.2)
    bin_centres, cum_hist = calc_cum_hist(ds_allS_c, "tot_hist", new_bin_centres)

    # plot seasonal cumulative histograms
    ax.axvline(4.5, color="gray", linestyle="-", linewidth=0.5, zorder=4)
    for i in range(len(ds_allS_c.season)):
        for i_lvl, lvl in enumerate(ds_allS_c.level):
            ax.plot(ds_allS_c.bin_centre.values, cum_hist[i, i_lvl, :-len(new_bin_centres)],
                    color=colors[i_lvl], alpha=0.4, zorder=2)
            ax.plot(np.concatenate(([4.5], new_bin_centres)), cum_hist[i, i_lvl, -len(new_bin_centres)-1:],
                    color=colors[i_lvl], alpha=0.1, zorder=2)

    threshold_vals = {}
    fit_res = calc_ppcf_fits(ds_allS_c, cum_hist, bin_centres)
    x_data_plt = np.arange(min(bin_centres), max(bin_centres), 0.01)
    for i_lvl, lvl in enumerate(ds_allS_c.level.data):
        if i_lvl < 5:
            y1 = logistic_gen(x_data_plt, *fit_res[lvl]["params_1"])
            y2 = logistic(x_data_plt, *fit_res[lvl]["params_2"])
            y_comb = y1 + y2
        else:
            y_comb = logistic_gen(x_data_plt, *fit_res[lvl]["params"])
        # plot fits per level
        ax.plot(x_data_plt, y_comb, ls="--", c="k", label="", linewidth=1.0, zorder=3)

        # calculate 1% of threshold per level
        threshold = 0.99 * fit_res[lvl]["Lpd"]
        idx = np.where(y_comb >= threshold)[0][0]
        threshold_vals[lvl] = (x_data_plt[idx], y_comb[idx])
        # ax.scatter(x_data_plt[idx], y_comb[idx], c="k", marker="o", s=8, zorder=5)

    # calculate and plot level-independent fit 
    y1t = logistic_gen(x_data_plt, *fit_res["all"]["params_1"])
    y2t = logistic(x_data_plt, *fit_res["all"]["params_2"])
    y_combt = y1t + y2t
    ax.plot(x_data_plt, y_combt, ls="-.", c="k", zorder=3)

    # calculate 1% of threshold for level-independent values
    threshold = 0.99 * fit_res["all"]["Lpd"]
    idx = np.where(y_combt >= threshold)[0][0]
    threshold_vals["all"] = (x_data_plt[idx], y_combt[idx])
    # ax.scatter(x_data_plt[idx], y_combt[idx], c="k", marker="o", s=8, zorder=5)
            
    # adjust limits
    ax.set_xlim([0, 6.5])
    ax.set_ylim([0, 0.2])
    ax.grid(visible=True, which='major', lw=0.1, c="gray")


# adjust axes and labels
ax_uncor.set_yticks(np.arange(0, 0.25, 0.05))
ax_95p.set_yticks(np.arange(0, 0.2, 0.05))

# create custom legend
legend_lines = [Line2D([0], [0], color=color, lw=2) for color in colors]
legend_lines.append(Line2D([0], [0], color="k", lw=1, ls="--"))
legend_lines.append(Line2D([0], [0], color="k", lw=1, ls="-."))
legend_labels = np.append(ds_allS_c.level.values, ["Fitted", "All data"])
ax_uncor.legend(legend_lines, legend_labels, loc="upper right", ncol=2, handlelength=3, framealpha=1)

plt.subplots_adjust(wspace=0, hspace=0)
fig.text(0.5, 0.04, "Mixing line slope $G$ [Pa/K]", ha="center", fontsize=12)
fig.text(0.04, 0.5, "Potential persistent contrail formation $p_{PCF}$ [-]", va='center', rotation='vertical', fontsize=12)

# add subplot labels
fig.text(0.14, 0.83, "a", fontsize=18)
fig.text(0.53, 0.83, "b", fontsize=18)
fig.text(0.14, 0.44, "c", fontsize=18)
fig.text(0.53, 0.44, "d", fontsize=18)

# save figure
# fig.savefig("figs/G-vs-ppcf_fitted_corr.png", dpi=600)